In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/24 17:07:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet("data/pq/green/*/*")

26/01/24 17:07:55 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/pq/green/*/*.
java.io.FileNotFoundException: File data/pq/green/*/* does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:980)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1301)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:970)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.sinks.FileStreamSink$.hasMetadata(FileStreamSink.scala:58)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:384)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.ResolveData

In [3]:
'''
SELECT 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    SUM(total_amount) AS amount,
    COUNT(1) as number_records

FROM 
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
'''

"\nSELECT \n    PULocationID AS zone,\n    date_trunc('hour', lpep_pickup_datetime) AS hour, \n    SUM(total_amount) AS amount,\n    COUNT(1) as number_records\n\nFROM \n    green\nWHERE\n    lpep_pickup_datetime >= '2020-01-01 00:00:00'\nGROUP BY\n    1,2\n"

In [3]:
df_green.rdd

MapPartitionsRDD[7] at javaToPython at NativeMethodAccessorImpl.java:0

In [4]:
rdd = df_green\
    .select('lpep_pickup_datetime','PULocationID','total_amount') \
    .rdd

In [11]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [13]:
rdd.filter(lambda row: False).take(1)

[]

In [5]:
from datetime import datetime

In [6]:
start = datetime(year=2020,month=1,day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [7]:
rdd \
    .filter(filter_outliers).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)]

In [19]:
rows = rdd.take(10)
row = rows[1]

In [26]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45)

In [8]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(second=0,minute=0,microsecond=0)
    zone = row.PULocationID
    key = (hour,zone)

    amount = row.total_amount
    count = 1
    value = (amount,count)

    return (key,value)

In [24]:
transform(row)

((datetime.datetime(2020, 1, 20, 15, 0), 67), (33.45, 1))

In [9]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count

    return (output_amount,output_count)
    

In [10]:
from collections import namedtuple

In [11]:
RevenueRow = namedtuple('RevenueRow',['hour','zone','revenue','count'])

In [12]:
def unwrap(row):
    return RevenueRow(hour=row[0][0],zone=row[0][1],revenue=row[1][0],count=row[1][1])

In [13]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .take(10)

[RevenueRow(hour=datetime.datetime(2020, 1, 4, 20, 0), zone=129, revenue=583.2700000000001, count=38),
 RevenueRow(hour=datetime.datetime(2020, 1, 3, 9, 0), zone=61, revenue=142.21, count=9),
 RevenueRow(hour=datetime.datetime(2020, 1, 12, 12, 0), zone=82, revenue=290.41, count=14),
 RevenueRow(hour=datetime.datetime(2020, 1, 28, 16, 0), zone=197, revenue=831.4399999999998, count=18),
 RevenueRow(hour=datetime.datetime(2020, 1, 10, 22, 0), zone=95, revenue=407.71000000000015, count=37),
 RevenueRow(hour=datetime.datetime(2020, 1, 18, 7, 0), zone=55, revenue=48.3, count=1),
 RevenueRow(hour=datetime.datetime(2020, 1, 12, 15, 0), zone=82, revenue=265.7900000000001, count=29),
 RevenueRow(hour=datetime.datetime(2020, 1, 31, 21, 0), zone=41, revenue=588.1600000000001, count=40),
 RevenueRow(hour=datetime.datetime(2020, 1, 25, 18, 0), zone=65, revenue=457.06000000000023, count=28),
 RevenueRow(hour=datetime.datetime(2020, 1, 26, 14, 0), zone=166, revenue=301.7900000000001, count=26)]

In [14]:
from pyspark.sql import types

In [15]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True), 
    types.StructField('zone', types.IntegerType(), True), 
    types.StructField('revenue', types.DoubleType(), True), 
    types.StructField('count', types.IntegerType(), True)
])

In [16]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [50]:
df_result.show()

[Stage 26:==================================================>       (7 + 1) / 8]

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-04 20:00:00| 129| 583.2700000000001|   38|
|2020-01-03 09:00:00|  61|            142.21|    9|
|2020-01-12 12:00:00|  82|            290.41|   14|
|2020-01-28 16:00:00| 197| 831.4399999999998|   18|
|2020-01-10 22:00:00|  95|407.71000000000015|   37|
|2020-01-18 07:00:00|  55|              48.3|    1|
|2020-01-12 15:00:00|  82| 265.7900000000001|   29|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-25 18:00:00|  65|457.06000000000023|   28|
|2020-01-26 14:00:00| 166| 301.7900000000001|   26|
|2020-01-28 15:00:00|  75|1745.2499999999984|   99|
|2020-01-23 12:00:00| 119|             51.89|    2|
|2020-01-17 15:00:00| 145|             96.25|    3|
|2020-01-12 00:00:00| 127|              17.8|    1|
|2020-01-24 16:00:00|  74|1072.2699999999995|   62|
|2020-01-04 10:00:00| 212|             81.67|    4|
|2020-01-03 

Traceback (most recent call last):                                              
  File "/opt/homebrew/Cellar/apache-spark/4.1.0/libexec/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
  File "/opt/homebrew/Cellar/apache-spark/4.1.0/libexec/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
    ~~~~~~~~~~~~~^^
BrokenPipeError: [Errno 32] Broken pipe


In [51]:
df_result.write.parquet('tmp/green-revenue')

26/01/21 22:42:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [22]:
columns = ['VendorID','lpep_pickup_datetime','PULocationID','DOLocationID','trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [25]:
duration_rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, DOLocationID=130, trip_distance=12.77),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, DOLocationID=39, trip_distance=8.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, DOLocationID=157, trip_distance=1.27),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, DOLocationID=83, trip_distance=1.25),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, DOLocationID=42, trip_distance=1.84)]

In [26]:
def apply_model_in_batch(partition):
    return [1]

In [28]:
rdd.mapPartitions(apply_model_in_batch).collect()

[1, 1, 1, 1, 1, 1, 1, 1]